<a href="https://colab.research.google.com/github/leoipp/CENIBRA-GEO/blob/main/Analise_de_Cobertura_de_Nuvem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importação de pacotes
import json
import pandas as pd

# Intalação de pacotes que nao conseguiram ser importados (Caso haja reinicilização do kernel, rode esta celula novamente)

try:
    import geemap, ee
    import seaborn as sns
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap seaborn matplotlib
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
        !conda install seaborn matplotlib -y
    import geemap, ee
    import seaborn as sns
    import matplotlib.pyplot as plt

In [2]:
# Inicialização e/ou Autenticação do Earth Engine 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=hx8Z4kmOWmmmWrvzc2OGpdzs6kbpt1v2CNDtU1k-30g&tc=m3YpThGa5pnJZnb27X9lKmiuZXauZ-DHIXR6DzPYZkE&cc=rwx7tV-upr29IEfSD293Zv3FxDxidHtnbD9lRhX2JcQ

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWg05Hb-6C0RmArfXlYLwMODBfPt1FzhwA0UVuL9zB0M0R1EdWn5XnM

Successfully saved authorization token.


In [98]:
#@markdown Indicação de parâmetros.
# Camada de área de interesse
AOI_GN = ee.Geometry.Polygon(
        [[[-43.45230715090038,-19.123106898376598],
          [-41.99936525636913,-19.123106898376598],
          [-41.99936525636913,-18.016604678857952],
          [-43.45230715090038,-18.016604678857952]]], None)

# Camada de área de interesse
AOI_NE = ee.Geometry.Polygon(
        [[[-43.56741785025536,-20.25056297245866],
          [-42.54843591666161,-20.25056297245866],
          [-42.54843591666161,-18.98807690624204],
          [-43.56741785025536,-18.98807690624204]]], None)

# Camada de área de interesse
AOI_RD = ee.Geometry.Polygon(
        [[[-42.69306006150326,-20.421412226789364],
          [-41.68231787400326,-20.421412226789364],
          [-41.68231787400326,-18.3332108430093],
          [-42.69306006150326,-18.3332108430093]]], None)


AOI_ALL = ee.Geometry.Polygon(
    [[[-43.39229632678435,-20.442183326873586],
      [-41.71138812365935,-20.442183326873586],
      [-41.71138812365935,-18.47412380535358],
      [-43.39229632678435,-18.47412380535358]]], None)

Regiao =  "ALL" #@param {type: "string"}
Data_inicio = "2017-01-01" #@param {type: "date"}
Data_final = "2021-12-20" #@param {type: "date"}

In [99]:
#@markdown Metadados de Probabilidade de Nuvens
# Importanção da feature collection para regiao de Rio Doce
region = ee.FeatureCollection('users/leoippef/BASE_AMOSTRAS_FOMENTO').filter(ee.Filter.eq('REGIAO_ID', Regiao))

def region_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=500, maxPixels=1e9).get('probability')
    return img.set('date', img.date().format()).set('prob', mean)


if Regiao == "RD":

  # Importação da coleção de imagens para regiao de Rio Doce
  s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filter(ee.Filter.bounds(AOI_RD)).filter(ee.Filter.date(ee.Date(Data_inicio), ee.Date(Data_final)))
  # Metadata da coleção de imagens adquirirdas no Google Earth Engine API.
  # Obter o range de datas com formato já convertido.
  range = s2Clouds.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
  Data_inicio = ee.Date(range.get('min')).format("YYYY-MM-dd")
  Data_final = ee.Date(range.get('max')).format("YYYY-MM-dd")
  print(f'Período de obtenção: Início em {Data_inicio.getInfo()} / Fim em {Data_final.getInfo()}')
  print(f'Para os parametros selecionados, você obteve: {s2Clouds.size().getInfo()} Imagens.')


  poi_reduced_imgs = s2Clouds.map(region_mean)
  nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date', 'prob']).values().get(0)
  df_rd = pd.DataFrame(nested_list.getInfo(), columns=['date', 'prob'])
  df_rd['date'] = pd.to_datetime(df_rd['date'])
  df_rd = df_rd.set_index('date')

if Regiao == "NE":

  # Importação da coleção de imagens para regiao de Rio Doce
  s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filter(ee.Filter.bounds(AOI_NE)).filter(ee.Filter.date(ee.Date(Data_inicio), ee.Date(Data_final)))
  # Metadata da coleção de imagens adquirirdas no Google Earth Engine API.
  # Obter o range de datas com formato já convertido.
  range = s2Clouds.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
  Data_inicio = ee.Date(range.get('min')).format("YYYY-MM-dd")
  Data_final = ee.Date(range.get('max')).format("YYYY-MM-dd")
  print(f'Período de obtenção: Início em {Data_inicio.getInfo()} / Fim em {Data_final.getInfo()}')
  print(f'Para os parametros selecionados, você obteve: {s2Clouds.size().getInfo()} Imagens.')

  poi_reduced_imgs = s2Clouds.map(region_mean)
  nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date', 'prob']).values().get(0)
  df_ne = pd.DataFrame(nested_list.getInfo(), columns=['date', 'prob'])
  df_ne['date'] = pd.to_datetime(df_ne['date'])
  df_ne = df_ne.set_index('date')

if Regiao == "GN":

  # Importação da coleção de imagens para regiao de Rio Doce
  s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filter(ee.Filter.bounds(AOI_GN)).filter(ee.Filter.date(ee.Date(Data_inicio), ee.Date(Data_final)))
  # Metadata da coleção de imagens adquirirdas no Google Earth Engine API.
  # Obter o range de datas com formato já convertido.
  range = s2Clouds.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
  Data_inicio = ee.Date(range.get('min')).format("YYYY-MM-dd")
  Data_final = ee.Date(range.get('max')).format("YYYY-MM-dd")
  print(f'Período de obtenção: Início em {Data_inicio.getInfo()} / Fim em {Data_final.getInfo()}')
  print(f'Para os parametros selecionados, você obteve: {s2Clouds.size().getInfo()} Imagens.')

  poi_reduced_imgs = s2Clouds.map(region_mean)
  nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date', 'prob']).values().get(0)
  df_gn = pd.DataFrame(nested_list.getInfo(), columns=['date', 'prob'])
  df_gn['date'] = pd.to_datetime(df_gn['date'])
  df_gn = df_gn.set_index('date')

if Regiao == "ALL":

  region = ee.FeatureCollection('users/leoippef/BASE_AMOSTRAS_FOMENTO')

  # Importação da coleção de imagens para regiao de Rio Doce
  s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filter(ee.Filter.bounds(AOI_ALL)).filter(ee.Filter.date(ee.Date(Data_inicio), ee.Date(Data_final)))
  # Metadata da coleção de imagens adquirirdas no Google Earth Engine API.
  # Obter o range de datas com formato já convertido.
  range = s2Clouds.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
  Data_inicio = ee.Date(range.get('min')).format("YYYY-MM-dd")
  Data_final = ee.Date(range.get('max')).format("YYYY-MM-dd")
  print(f'Período de obtenção: Início em {Data_inicio.getInfo()} / Fim em {Data_final.getInfo()}')
  print(f'Para os parametros selecionados, você obteve: {s2Clouds.size().getInfo()} Imagens.')

  poi_reduced_imgs = s2Clouds.map(region_mean)
  nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date', 'prob']).values().get(0)
  df_all = pd.DataFrame(nested_list.getInfo(), columns=['date', 'prob'])
  df_all['date'] = pd.to_datetime(df_all['date'])
  df_all = df_all.set_index('date')

Período de obtenção: Início em 2017-01-06 / Fim em 2021-12-19
Para os parametros selecionados, você obteve: 6621 Imagens.


In [112]:
fig = px.line(df_all, x=df_all.index, y=df_all['prob'].rolling(90).mean(), title='Probabilidade de Cobertura de Nuvens',
              labels={'date':"Ano",
                      'y': 'Probabilidade (%)'})
fig.show()

In [102]:
print(df_all.idxmax())
print(df_all.idxmin())

prob   2017-02-01 13:12:41
dtype: datetime64[ns]
prob   2021-03-27 13:12:39
dtype: datetime64[ns]
